In [1]:
# -*- coding: utf-8 -*-
"""
Diese Datei sollte nicht verändert werden und wird von uns gestellt und zurückgesetzt.

Funktionen zum Laden und Speichern der Dateien
"""
__author__ = "Maurice Rohr und Dirk Schweickard"

from typing import List, Tuple, Dict, Any
import csv
import scipy.io as sio
import numpy as np
import os


### Achtung! Diese Funktion nicht veraendern.
def load_references(folder: str = '../../training') -> Tuple[List[str], List[List[str]],
                                                          List[np.ndarray],  List[float],
                                                          List[str], List[Tuple[bool,float,float]]]:
    """
    Liest Referenzdaten aus .mat (Messdaten) und .csv (Label) Dateien ein.
    Parameters
    ----------
    folder : str, optional
        Ort der Trainingsdaten. Default Wert '../training'.

    Returns
    -------
    ids : List[str]
        Liste von ID der Aufnahmen
    channels : List[List[str]]
        Liste der vorhandenen Kanäle per Aufnahme
    data :  List[ndarray]
        Liste der Daten pro Aufnahme
    sampling_frequencies : List[float]
        Liste der Sampling-Frequenzen.
    reference_systems : List[str]
        Liste der Referenzsysteme. "LE", "AR", "Sz" (Zusatz-Information)
    """
    
    # Initialisiere Listen ids, channels, data, sampling_frequencies, refernece_systems und eeg_labels
    ids: List[str] = []
    channels: List[List[str]] = []
    data: List[np.ndarray] = []
    sampling_frequencies: List[float] = []
    reference_systems: List[str] = []
    eeg_labels: List[Tuple[bool,float,float]] = []
    
    # Erzeuge Datensatz aus Ordner und fülle Listen mit Daten
    dataset = EEGDataset(folder)
    for item in dataset:
        ids.append(item[0])
        channels.append(item[1])
        data.append(item[2])
        sampling_frequencies.append(item[3])
        reference_systems.append(item[4])
        eeg_labels.append(item[5])
        
    # Zeige an wie viele Daten geladen wurden
    print("{}\t Dateien wurden geladen.".format(len(ids)))
    return ids, channels, data, sampling_frequencies, reference_systems, eeg_labels

### Achtung! Diese Klasse nicht veraendern.
class EEGDataset:
    def __init__(self,folder:str) -> None:
        """Diese Klasse stellt einen EEG Datensatz dar.
        
        Verwendung:
            Erzeuge einen neuen Datensatz (ohne alle Daten zu laden) mit
            dataset = EEGDataset("../training/")
            len(dataset) # gibt Größe des Datensatzes zurück
            dataset[0] # gibt erstes Element aus Datensatz zurück bestehend aus (id, channels, data, sampling_frequency, reference_system, eeg_label)
            it = iter(dataset) # gibt einen iterator zurück auf den Datensatz,
            next(it) # gibt nächstes Element zurück bis alle Daten einmal geholt wurden
            for item in dataset: # iteriert einmal über den gesamten Datensatz
                (id, channels, data, sampling_frequency, reference_system, eeg_label) = item
                # Berechnung

        Args:
            folder (str): Ordner in dem der Datensatz bestehend aus .mat-Dateien und einer REFERENCE.csv Datei liegt
        """
        assert isinstance(folder, str), "Parameter folder muss ein string sein aber {} gegeben".format(type(folder))
        assert os.path.exists(folder), 'Parameter folder existiert nicht!'
        # Initialisiere Listen für ids und labels
        self._folder = folder
        self._ids: List[str] = []
        self._eeg_labels: List[Tuple[bool,float,float]] = []
        # Lade references Datei
        with open(os.path.join(folder, 'REFERENCE.csv')) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            # Iteriere über jede Zeile
            for row in csv_reader:
                self._ids.append(row[0])
                self._eeg_labels.append((int(row[1]),float(row[2]),float(row[3])))
    
    def __len__(self):
        return len(self._ids)
    
    def __getitem__(self,idx) -> Tuple[str, List[str],
                                    np.ndarray,  float,
                                    str, Tuple[bool,float,float]]:
        #Lade Matlab-Datei
        eeg_data = sio.loadmat(os.path.join(self._folder, self._ids[idx] + '.mat'),simplify_cells=True)
        ch_names = eeg_data.get('channels')
        channels = [x.strip(' ') for x in ch_names] 
        data = eeg_data.get('data')
        sampling_frequency = eeg_data.get('fs')
        reference_system = eeg_data.get('reference_system')
        return (self._ids[idx],channels,data,sampling_frequency,reference_system,self._eeg_labels[idx])
    
    def get_labels(self):
        return self._eeg_labels
    
        
    
        




### Achtung! Diese Funktion nicht veraendern.
#predictions = {"id":id,"seizure_present":seizure_present,"seizure_confidence":seizure_confidence,
#                   "onset":onset,"onset_confidence":onset_confidence,"offset":offset,
#                   "offset_confidence":offset_confidence}
def save_predictions(predictions: List[Dict[str,Any]], folder: str=None) -> None:
    """
    Funktion speichert the gegebenen predictions in eine CSV-Datei mit dem name PREDICTIONS.csv. 
    Alle Optionalen Vorherhsagen werden mit Standardwerten ersetzt.
    Parameters
    ----------
    predictions : List[Dict[str,Any]]
        Liste aus dictionaries. Jedes Dictionary enthält die Felder "id","seizure_present",
                "seizure_confidence" (optional),"onset","onset_confidence" (optional),
                "offset" (optional),"offset_confidence" (optional)
	folder : str
		Speicherort der predictions
    Returns
    -------
    None.

    """    
	# Check Parameter
    assert isinstance(predictions, list), \
        "Parameter predictions muss eine Liste sein aber {} gegeben.".format(type(predictions))
    assert len(predictions) > 0, 'Parameter predictions muss eine nicht leere Liste sein.'
    assert isinstance(predictions[0], dict), \
        "Elemente der Liste predictions muss ein Dictionary sein aber {} gegeben.".format(type(predictions[0]))
    assert "id" in predictions[0], \
        "Prädiktionen müssen eine ID besitzen, aber Key in Dictionary nicht vorhanden"
	
    if folder==None:
        file = "PREDICTIONS.csv"
    else:
        file = os.path.join(folder, "PREDICTIONS.csv")
    # Check ob Datei schon existiert wenn ja loesche Datei
    if os.path.exists(file):
        os.remove(file)

    with open(file, mode='w', newline='') as predictions_file:

        # Init CSV writer um Datei zu beschreiben
        predictions_writer = csv.writer(predictions_file, delimiter=',')
        # Iteriere über jede prediction
        header=["id","seizure_present","seizure_confidence","onset","onset_confidence","offset","offset_confidence"]
        predictions_writer.writerow(header)
        for prediction in predictions:
            _id = prediction["id"]
            _seizure_present = prediction["seizure_present"]
            _seizure_confidence = prediction.get("seizure_confidence",1.0) 
            _onset = prediction["onset"]
            _onset_confidence = prediction.get("onset_confidence",1.0) 
            _offset = prediction.get("offset",999999.0)
            _offset_confidence = prediction.get("offset_confidence",0.0)
            predictions_writer.writerow([_id,_seizure_present,_seizure_confidence,_onset,_onset_confidence,_offset,_offset_confidence])
        # Gebe Info aus wie viele labels (predictions) gespeichert werden
        print("{}\t Labels wurden geschrieben.".format(len(predictions)))
        

def get_3montages(channels: List[str], data: np.ndarray) -> Tuple[List[str],np.ndarray,bool]:
    """
    Funktion berechnet die 3 Montagen Fp1-F3, Fp2-F4, C3-P3 aus den gegebenen Ableitungen (Montagen)
    zur selben Referenzelektrode. Falls nicht alle nötigen Elektroden vorhanden sind, wird das entsprechende Signal durch 0 ersetzt. 
    ----------
    channels : List[str]
        Namen der Kanäle z.B. Fp1, Cz, C3
	data : ndarray
		Daten der Kanäle
    Returns
    -------
    montages : List[str]
        Namen der Montagen ["Fp1-F3", "Fp2-F4", "C3-P3"]
    montage_data : ndarray
        Daten der Montagen
    montage_missing : bool
        1 , falls eine oder mehr Montagen fehlt, sonst 0

    """   
    montages = []
    _,m = np.shape(data)
    montage_data = np.zeros([3,m])
    montage_missing = 0
    if '-' in channels:
        try:
            montage_data[0,:] = data[channels.index('Fp1-F3')]
            montages.append('Fp1-F3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[1,:] = data[channels.index('Fp2-F4')]
            montages.append('Fp2-F4')
        except:
            montage_missing = 1
            montages.append('error')        
        try:
            montage_data[2,:] = data[channels.index('C3-P3')]
            montages.append('C3-P3')
        except:
            montage_missing = 1
            montages.append('error')

        return (montages,montage_data,montage_missing)

    else:
        try:
            montage_data[0,:] = data[channels.index('Fp1')] - data[channels.index('F3')]
            montages.append('Fp1-F3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[1,:] = data[channels.index('Fp2')] - data[channels.index('F4')]
            montages.append('Fp2-F4')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[2,:] = data[channels.index('C3')] - data[channels.index('P3')]
            montages.append('C3-P3')
        except:
            montage_missing = 1
            montages.append('error')

        return (montages,montage_data,montage_missing)


def get_6montages(channels: List[str], data: np.ndarray) -> Tuple[List[str],np.ndarray,bool]:
    """
    Funktion berechnet die 6 Montagen Fp1-F3, Fp2-F4, C3-P3, F3-C3, F4-C4, C4-P4 aus den gegebenen Ableitungen (Montagen)
    zur selben Referenzelektrode. Falls nicht alle nötigen Elektroden vorhanden sind, wird das entsprechende Signal durch 0 ersetzt. 
    ----------
    channels : List[str]
        Namen der Kanäle z.B. Fp1, Cz, C3
	data : ndarray
		Daten der Kanäle
    Returns
    -------
    montages : List[str]
        Namen der Montagen ["Fp1-F3", "Fp2-F4", "C3-P3", "F3-C3", "F4-C4", "C4-P4"]
    montage_data : ndarray
        Daten der Montagen
    montage_missing : bool
        1 , falls eine oder mehr Montagen fehlt, sonst 0

    """  
    montages = []
    _,m = np.shape(data)
    montage_data = np.zeros([6,m])
    montage_missing = 0
    if '-' in channels:
        try:
            montage_data[0,:] = data[channels.index('Fp1-F3')]
            montages.append('Fp1-F3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[1,:] = data[channels.index('Fp2-F4')]
            montages.append('Fp2-F4')
        except:
            montage_missing = 1
            montages.append('error')        
        try:
            montage_data[2,:] = data[channels.index('C3-P3')]
            montages.append('C3-P3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[3,:] = data[channels.index('F3-C3')]
            montages.append('F3-C3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[4,:] = data[channels.index('F4-C4')]
            montages.append('F4-C4')
        except:
            montage_missing = 1
            montages.append('error')        
        try:
            montage_data[5,:] = data[channels.index('C4-P4')]
            montages.append('C4-P4')
        except:
            montage_missing = 1
            montages.append('error')

        return (montages,montage_data,montage_missing)

    else:         
        try:
            montage_data[0,:] = data[channels.index('Fp1')] - data[channels.index('F3')]
            montages.append('Fp1-F3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[1,:] = data[channels.index('Fp2')] - data[channels.index('F4')]
            montages.append('Fp2-F4')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[2,:] = data[channels.index('C3')] - data[channels.index('P3')]
            montages.append('C3-P3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[3,:] = data[channels.index('F3')] - data[channels.index('C3')]
            montages.append('F3-C3')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[4,:] = data[channels.index('F4')] - data[channels.index('C4')]
            montages.append('F4-C4')
        except:
            montage_missing = 1
            montages.append('error')
        try:
            montage_data[5,:] = data[channels.index('C4')] - data[channels.index('P4')]
            montages.append('C4-P4')
        except:
            montage_missing = 1
            montages.append('error')

        return (montages,montage_data,montage_missing)


In [2]:
# -*- coding: utf-8 -*-
"""

Skript testet das vortrainierte Modell


@author:  Maurice Rohr, Dirk Schweickard
"""


import numpy as np
import json
import os
from typing import List, Tuple, Dict, Any
from wettbewerb import get_3montages

# Pakete aus dem Vorlesungsbeispiel
import mne
from scipy import signal as sig
import ruptures as rpt

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from wettbewerb import load_references, get_3montages, get_6montages
import mne
from scipy import signal as sig
from imblearn.under_sampling import RandomUnderSampler

###Signatur der Methode (Parameter und Anzahl return-Werte) darf nicht verändert werden
def predict_labels(channels : List[str], data : np.ndarray, fs : float, reference_system: str, model_name : str='checkpoint.pt') -> Dict[str,Any]:
    '''
    Parameters
    ----------
    channels : List[str]
        Namen der übergebenen Kanäle
    data : ndarray
        EEG-Signale der angegebenen Kanäle
    fs : float
        Sampling-Frequenz der Signale.
    reference_system :  str
        Welches Referenzsystem wurde benutzt, "Bezugselektrode", nicht garantiert korrekt!
    model_name : str
        Name eures Models,das ihr beispielsweise bei Abgabe genannt habt. 
        Kann verwendet werden um korrektes Model aus Ordner zu laden
    Returns
    -------
    prediction : Dict[str,Any]
        enthält Vorhersage, ob Anfall vorhanden und wenn ja wo (Onset+Offset)
    '''

#------------------------------------------------------------------------------
# Euer Code ab hier  

    # Initialisiere Return (Ergebnisse)
    seizure_present = True # gibt an ob ein Anfall vorliegt
    seizure_confidence = 0.5 # gibt die Unsicherheit des Modells an (optional)
    onset = 4.2   # gibt den Beginn des Anfalls an (in Sekunden)
    onset_confidence = 0.99 # gibt die Unsicherheit bezüglich des Beginns an (optional)
    offset = 999999  # gibt das Ende des Anfalls an (optional)
    offset_confidence = 0   # gibt die Unsicherheit bezüglich des Endes an (optional)


# CNN-Modell
    class CNN(nn.Module):
        def __init__(self, num_classes, seq_length):
            super().__init__()
            self.classifier = nn.Sequential(
                nn.Conv1d(in_channels=3, out_channels=6, kernel_size=5),
                nn.BatchNorm1d(num_features=6),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=2, stride=2),
                nn.Conv1d(6, 16, 5),
                nn.BatchNorm1d(16),
                nn.ReLU(),
                nn.MaxPool1d(2, 2),
            )

            # Anpassung für die Berechnung der Größe des linearen Layers
            linear_input_size = self._get_conv_output(seq_length)

            self.fc = nn.Sequential(
                nn.Linear(linear_input_size, 120),
                nn.BatchNorm1d(120),
                nn.ReLU(),
                nn.Linear(120, 84),
                nn.BatchNorm1d(84),
                nn.ReLU(),
                nn.Linear(84, num_classes),
            )

        def _get_conv_output(self, shape):
            with torch.no_grad():
                input = torch.zeros(1, 3, shape)
                output = self.classifier(input)
                return output.numel()

        def forward(self, x):
            x = self.classifier(x)
            x = torch.flatten(x, start_dim=1)
            x = self.fc(x)
            return x
    
    cnn_classifier = CNN(num_classes=2, seq_length=2000)
    checkpoint = torch.load(model_name)
    cnn_classifier.load_state_dict(checkpoint)
    cnn_classifier.eval()

    
    number_montages = 3
    N_samples = 2000 # Number of samples per division
    # Decompose the wave
    wavelet = 'db4'
    scaler = StandardScaler()
    new_signal = []

    mont1_signal = []
    mont2_signal = []
    mont3_signal = []
    whole_mont = [mont1_signal,mont2_signal,mont3_signal]
    for i,_id in enumerate(ids):
    
        if number_montages == 6:
            _montage, _montage_data, _is_missing = get_6montages(channels[i], data[i])
        else:
            _montage, _montage_data, _is_missing = get_3montages(channels[i], data[i])
        
        _fs = sampling_frequencies[i]
        features_per_id = []

        for j, signal_name in enumerate(_montage):
            signal = _montage_data[j]
            # Notch-Filter to compensate net frequency of 50 Hz
            signal_notch = mne.filter.notch_filter(x=signal, Fs=_fs, freqs=np.array([50.,100.]), n_jobs=2, verbose=False)
            # Bandpassfilter between 0.5Hz and 70Hz to filter out noise
            signal_filter = mne.filter.filter_data(data=signal_notch, sfreq=_fs, l_freq=0.5, h_freq=70.0, n_jobs=2, verbose=False)
            # Defining number of divisions for signal
            N_div = len(signal_filter)//N_samples
            # Normalizing data
            norm_montage_data = scaler.fit_transform(signal_filter.reshape(-1,1)).reshape(1,-1)[0]
    
            for i in range(N_div):
                montage_array = norm_montage_data[i*N_samples:(i+1)*N_samples]
                whole_mont[j].append(montage_array)


    labels = []
    for i,_id in enumerate(ids):
        if eeg_labels[i][0]:
            onset = eeg_labels[i][1]
            offset = eeg_labels[i][2]
            sample_freq = sampling_frequencies[i]
            total_time = len(data[i][1])/sample_freq
            N_div = len(data[i][1])//N_samples
            for num in range(N_div):
                if (((total_time/N_div)*(num) <= onset) and ((total_time/N_div)*(num+1) > onset)) or (((total_time/N_div)*(num) >= onset) and ((total_time/N_div)*(num) < offset)):
                    labels.append([1])
                else:
                    labels.append([0])
        else:
            N_div = len(data[i][1])//N_samples
            for num in range(N_div):
                labels.append([0])
    labels = np.reshape(labels, (1,-1))[0]


    # Instanziierung von RandomUnderSampler
    undersample = RandomUnderSampler()

    # Erstellen Sie eine Funktion, die das Resampling durchführt
    def resample_signal(signal, labels):
        # Anwenden von RandomUnderSampler
        signal_resampled, labels_resampled = undersample.fit_resample(signal, labels)
        return signal_resampled, labels_resampled

    # Anwenden der Funktion auf jedes Signal
    mont1_signal_resampled, labels_resampled_1 = resample_signal(np.array(mont1_signal), labels)
    mont2_signal_resampled, labels_resampled_2 = resample_signal(np.array(mont2_signal), labels)
    mont3_signal_resampled, labels_resampled_3 = resample_signal(np.array(mont3_signal), labels)

    # Stellen Sie sicher, dass die Labels für alle Signale gleich sind, da sie das gleiche Set von Beispielen repräsentieren sollten
    assert np.array_equal(labels_resampled_1, labels_resampled_2)
    assert np.array_equal(labels_resampled_1, labels_resampled_3)

    labels_resampled = labels_resampled_1

    whole_mont_resampled = [mont1_signal_resampled,mont2_signal_resampled,mont3_signal_resampled]


    whole_mont_resampled_np = np.array(whole_mont_resampled)


    # Konvertieren der Daten in PyTorch Tensoren und Vorbereiten für das Modell
    data_tensor = torch.tensor(whole_mont_resampled_np, dtype=torch.float).permute(1, 0, 2)  # Permutieren zu [Anzahl der Beispiele, Kanäle, Länge]

    # Stellen Sie sicher, dass Ihr Modell und Daten auf dem gleichen Gerät sind
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_classifier = cnn_classifier.to(device)
    data_tensor = data_tensor.to(device)

    # Bereiten Sie die Struktur für die Erfassung der Vorhersagen vor
    seizure_present_array = np.zeros(data_tensor.shape[0], dtype=bool)

    # Vorhersagen durchführen und überprüfen, ob ein Anfall vorhanden ist
    with torch.no_grad():
        predictions = cnn_classifier(data_tensor)  # Vorhersagen für das gesamte Batch
        predicted_classes = torch.argmax(predictions, dim=1)
        seizure_indices = torch.where(predicted_classes == 1)[0]  # Angenommen, Klasse 1 steht für "seizure"

        if len(seizure_indices) > 0:
            seizure_present = True
            # Erster Anfallindex kann als Anfallsbeginn betrachtet werden
            first_seizure_index = seizure_indices[0].item()
            onset = N_samples * first_seizure_index / fs  # Berechnung des Anfallsbeginns in Sekunden
            offset = N_samples * seizure_indices[-1].item() / fs  # Berechnung des Anfallsendes
            # Annahme: Confidence-Werte sind statisch, könnten dynamisch angepasst werden
            seizure_confidence = 0.8  # Beispielwert, anpassen basierend auf Ihrer Modellausgabe oder Heuristik
            offset_confidence = 0.8  # Beispielwert, anpassen basierend auf Analyse


    
 

     
     
    
#------------------------------------------------------------------------------  
    prediction = {"seizure_present":seizure_present,"seizure_confidence":seizure_confidence,
                   "onset":onset,"onset_confidence":onset_confidence,"offset":offset,
                   "offset_confidence":offset_confidence}
  
    return prediction # Dictionary mit prediction - Muss unverändert bleiben!
                               
                               
        


In [3]:
# -*- coding: utf-8 -*-
"""
Diese Datei sollte nicht verändert werden und wird von uns gestellt und zurückgesetzt.

Skript testet das vortrainierte Modell


@author: Maurice Rohr
"""


from predict_cnn import predict_labels
from wettbewerb import EEGDataset, save_predictions
import argparse
import time

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Predict given Model')
    parser.add_argument('--test_dir', action='store',type=str,default='../../test/')
    parser.add_argument('--model_name', action='store',type=str,default='model.json')
    parser.add_argument('--allow_fail',action='store_true',default=False)
    args = parser.parse_args()
    
    # Erstelle EEG Datensatz aus Ordner
    dataset = EEGDataset(args.test_dir)
    print(f"Teste Modell auf {len(dataset)} Aufnahmen")
    
    predictions = list()
    start_time = time.time()
    
    # Rufe Predict Methode für jedes Element (Aufnahme) aus dem Datensatz auf
    for item in dataset:
        id,channels,data,fs,ref_system,eeg_label = item
        try:
            _prediction = predict_labels(channels,data,fs,ref_system,model_name=args.model_name)
            _prediction["id"] = id
            predictions.append(_prediction)
        except:
            if args.allow_fail:
                raise
        
    pred_time = time.time()-start_time
    
    save_predictions(predictions) # speichert Prädiktion in CSV Datei
    print("Runtime",pred_time,"s")


usage: ipykernel_launcher.py [-h] [--test_dir TEST_DIR]
                             [--model_name MODEL_NAME] [--allow_fail]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/jupyter-wki_team_6/.local/share/jupyter/runtime/kernel-aa4a5be9-86b8-493e-8cf6-dc1c344b273a.json


SystemExit: 2

/opt/tljh/user/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# Bereiten Sie die Struktur für die Erfassung der Vorhersagen vor
    seizure_present_array = np.zeros(data_tensor.shape[0], dtype=bool)

    # Vorhersagen durchführen und überprüfen, ob ein Anfall vorhanden ist
    with torch.no_grad():
        predictions = cnn_classifier(data_tensor)  # Vorhersagen für das gesamte Batch
        predicted_classes = torch.argmax(predictions, dim=1)
        seizure_indices = torch.where(predicted_classes == 1)[0]  # Angenommen, Klasse 1 steht für "seizure"

        if len(seizure_indices) > 0:
            seizure_present = [1]
            # Erster Anfallindex kann als Anfallsbeginn betrachtet werden
            first_seizure_index = seizure_indices[0].item()
            onset = N_samples * first_seizure_index / fs  # Berechnung des Anfallsbeginns in Sekunden
            offset = N_samples * seizure_indices[-1].item() / fs  # Berechnung des Anfallsendes
            # Annahme: Confidence-Werte sind statisch, könnten dynamisch angepasst werden
            seizure_confidence = 0.8  # Beispielwert, anpassen basierend auf Ihrer Modellausgabe oder Heuristik
            offset_confidence = 0.8  # Beispielwert, anpassen basierend auf Analyse

IndentationError: unexpected indent (781199126.py, line 2)

In [ ]:
python predict_pretrained.py --test_dir ../../test/

In [ ]:
python score.py --test_dir ../../test/